# Estimating chances of winning a lottery

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49](https://en.wikipedia.org/wiki/Lotto_6/49) lottery and build functions that enable users to answer questions like:

 - What is the probability of winning the big prize with a single ticket?
 - What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
 - What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. [The data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018 (we'll come back to this).

## Write functions to compute factorials and permutations

In [3]:
# function to calculate factorial
def factorial(n):
    f = 1
    for i in range (n, 0, -1):
        f *= i
    return f

In [4]:
# function to calculate denominator
def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial((n-k))
    return numerator/denominator

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. 

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

Inside the app, the user inputs six different numbers from 1 to 49.
Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.


In [7]:
def one_ticket_probability(array):
    num_comb = combinations(49, 6)
    p = 1 / num_comb
    return 'There is a {:f}% chance to win with that ticket.'.format((p * 100))

In [51]:
array_user = input('Please enter 6 numbers: ')
print(one_ticket_probability(array_user))

Please enter 6 numbers: 54 03 58 95 7 8
There is a 0.000007% chance to win with that ticket.


## Historical data on 6/49 lottery 

For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

On this screen, we'll focus on exploring the historical data coming from the Canada 6/49 lottery. 
The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn in the following six columns:

 - `NUMBER DRAWN 1`
 - `NUMBER DRAWN 2`
 - `NUMBER DRAWN 3`
 - `NUMBER DRAWN 4`
 - `NUMBER DRAWN 5`
 - `NUMBER DRAWN 6`
 
Let's now write some code to open this data set and get familiar with its structure.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
lottery = pd.read_csv('649.csv')
lottery.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [12]:
lottery.tail()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3660,649,3587,0,6/6/2018,10,15,23,38,40,41,35
3661,649,3588,0,6/9/2018,19,25,31,36,46,47,26
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


In [13]:
lottery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT          3665 non-null   int64 
 1   DRAW NUMBER      3665 non-null   int64 
 2   SEQUENCE NUMBER  3665 non-null   int64 
 3   DRAW DATE        3665 non-null   object
 4   NUMBER DRAWN 1   3665 non-null   int64 
 5   NUMBER DRAWN 2   3665 non-null   int64 
 6   NUMBER DRAWN 3   3665 non-null   int64 
 7   NUMBER DRAWN 4   3665 non-null   int64 
 8   NUMBER DRAWN 5   3665 non-null   int64 
 9   NUMBER DRAWN 6   3665 non-null   int64 
 10  BONUS NUMBER     3665 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 315.1+ KB


To enable a comparison with historical data the engineering team wants us to write a function that prints:
 - the number of times the combination selected occurred in the Canada data set
 - the probability of winning the big prize in the next drawing with that combination.

In [44]:
# fucntion to extract 6 numbers for every draw
def extract_numbers(row):
    nmb = row[4:10].to_numpy()
    set_nmb = set(nmb)
    return set_nmb

nmb = lottery.apply(extract_numbers, axis=1)
nmb.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [52]:
def check_historical_occurence(array_user, nmb):
    set_user = set(array_user)
    matches = nmb[nmb == set_user].sum()
    return matches

matches = check_historical_occurence(array_user, nmb)

In [54]:
print('Starting from June 1982 up to June 2018 there was {} winning tickets with your combination.'.format(matches))

Starting from June 1982 up to June 2018 there was 0 winning tickets with your combination


Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

 - The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
 - Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
 - The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [75]:
def multi_ticket_probability(n_tickets):
    comb = combinations(49, 6)
    n_outcomes = n_tickets
    prob = n_outcomes/comb
    return 'There is a {:f}% chance to win with that ticket.'.format((prob * 100))
n_tickets = int(input('How much tickets do you play with?'))

How much tickets do you play with?5000


In [76]:
print(multi_ticket_probability(n_tickets))

There is a 0.035756% chance to win with that ticket.


On this screen, we're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

 - Inside the app, the user inputs:
                               - six different numbers from 1 to 49; and
                               - an integer between 2 and 5 that represents the number of winning numbers expected
                                
Our function prints information about the probability of having the inputted number of winning numbers.

To help you code this function, we'll guide you through calculating the probability for having exactly five winning numbers. First, we need to differentiate between these two probability questions:

What is the probability of having exactly five winning numbers?
What is the probability of having at least five winning numbers?

In [77]:
def probability_less_6(n_matches):
    comb_ticket = combinations(6, n_matches)
    comb_draw = 49 - comb_ticket
    total_outcomes = combinations(49, 6)
    prob = (comb_ticket+comb_draw)/total_outcomes
    return 'There is a {:f}% chance to win with combination of {} numbers.'.format((prob * 100), n_matches)

In [78]:
n_matches = int(input('How many matches would you expect to have?'))

How many matches would you expect to have?4


In [79]:
print(probability_less_6(n_matches))

There is a 0.000350% chance to win with combination of 4 numbers.


Possible features for a second version of the app include:

Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery").
Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time.
Creating a function similar to probability_less_6() which calculates the probability of having at least two, three, four or five winning numbers. Hint: the number of successful outcomes for having at least four winning numbers is the sum of these three numbers:
The number of successful outcomes for having four winning numbers exactly
The number of successful outcomes for having five winning numbers exactly
The number of successful outcomes for having six winning numbers exactly